In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#importing necessary packages
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import tflearn
import tensorflow as tf
from PIL import Image
%matplotlib inline
#for writing text files
import glob
import os     
import random 
#reading images from a text file
from tflearn.data_utils import image_preloader
import math

In [ ]:
IMAGE_FOLDER = 'C:/Users/drago/DvsG/train/train'
TRAIN_DATA = 'C:/Users/drago/DvsG/training_data.txt'
TEST_DATA = 'C:/Users/drago/DvsG/test_data.txt'
VALIDATION_DATA = 'C:/Users/drago/DvsG/validation_data.txt'
train_proportion=0.7
test_proportion=0.2
validation_proportion=0.1

In [ ]:
#read the image directories
filenames_image = os.listdir(IMAGE_FOLDER)
#shuffling the data is important otherwise the model will be fed with a single class data for a long time and 
#network will not learn properly
random.shuffle(filenames_image)
#total number of images
total=len(filenames_image)
##  *****training data******** 
fr = open(TRAIN_DATA, 'w')
train_files=filenames_image[0: int(train_proportion*total)]
for filename in train_files:
    if filename[0:3] == 'cat':
        fr.write(IMAGE_FOLDER + '/'+ filename + ' 0\n')
    elif filename[0:3] == 'dog':
        fr.write(IMAGE_FOLDER + '/'+ filename + ' 1\n')

fr.close()
##  *****testing data******** 
fr = open(TEST_DATA, 'w')
test_files=filenames_image[int(math.ceil(train_proportion*total)):int(math.ceil((train_proportion+test_proportion)*total))]
for filename in test_files:
    if filename[0:3] == 'cat':
        fr.write(IMAGE_FOLDER + '/'+ filename + ' 0\n')
    elif filename[0:3] == 'dog':
        fr.write(IMAGE_FOLDER + '/'+ filename + ' 1\n')
fr.close()

##  *****validation data******** 
fr = open(VALIDATION_DATA, 'w')
valid_files=filenames_image[int(math.ceil((train_proportion+test_proportion)*total)):total]
for filename in valid_files:
    if filename[0:3] == 'cat':
        fr.write(IMAGE_FOLDER + '/'+ filename + ' 0\n')
    elif filename[0:3] == 'dog':
        fr.write(IMAGE_FOLDER + '/'+ filename + ' 1\n')
fr.close()

In [ ]:
#Importing data
#importing data
X_train, Y_train = image_preloader(TRAIN_DATA, image_shape=(56,56),mode='file', categorical_labels=True,normalize=True)
X_test, Y_test = image_preloader(TEST_DATA, image_shape=(56,56),mode='file', categorical_labels=True,normalize=True)
X_val, Y_val = image_preloader(VALIDATION_DATA, image_shape=(56,56),mode='file', categorical_labels=True,normalize=True)

In [ ]:
print ("Dataset")
print ("Number of training images {}".format(len(X_train)))
print ("Number of testing images {}".format(len(X_test)))
print ("Number of validation images {}".format(len(X_val)))
print ("Shape of an image {}" .format(X_train[1].shape))
print ("Shape of label:{} ,number of classes: {}".format(Y_train[1].shape,len(Y_train[1])))

In [ ]:
#Sample Image 
plt.imshow(X_train[1])
plt.axis('off')
plt.title('Sample image with label {}'.format(Y_train[1]))
plt.show()

In [ ]:
x=tf.placeholder(tf.float32,shape=[None,56,56,3] , name='input_image') 
#input class
y_=tf.placeholder(tf.float32,shape=[None, 2] , name='input_class')

In [ ]:
input_layer=x
#convolutional layer 1 --convolution+RELU activation
conv_layer1=tflearn.layers.conv.conv_2d(input_layer, nb_filter=64, filter_size=5, strides=[1,1,1,1],
                                        padding='same', activation='relu', regularizer="L2", name='conv_layer_1')

#2x2 max pooling layer
out_layer1=tflearn.layers.conv.max_pool_2d(conv_layer1, 2)


#second convolutional layer 
conv_layer2=tflearn.layers.conv.conv_2d(out_layer1, nb_filter=128, filter_size=5, strides=[1,1,1,1],
                                        padding='same', activation='relu',  regularizer="L2", name='conv_layer_2')
out_layer2=tflearn.layers.conv.max_pool_2d(conv_layer2, 2)
# third convolutional layer
conv_layer3=tflearn.layers.conv.conv_2d(out_layer2, nb_filter=128, filter_size=5, strides=[1,1,1,1],
                                        padding='same', activation='relu',  regularizer="L2", name='conv_layer_2')
out_layer3=tflearn.layers.conv.max_pool_2d(conv_layer3, 2)

#fully connected layer1
fcl= tflearn.layers.core.fully_connected(out_layer3, 4096, activation='relu' , name='FCL-1')
fcl_dropout_1 = tflearn.layers.core.dropout(fcl, 0.8)
#fully connected layer2
fc2= tflearn.layers.core.fully_connected(fcl_dropout_1, 4096, activation='relu' , name='FCL-2')
fcl_dropout_2 = tflearn.layers.core.dropout(fc2, 0.8)
#softmax layer output
y_predicted = tflearn.layers.core.fully_connected(fcl_dropout_2, 2, activation='softmax', name='output')

In [ ]:
#loss function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_predicted+np.exp(-10)), reduction_indices=[1]))
#optimiser -
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#calculating accuracy of our model 
correct_prediction = tf.equal(tf.argmax(y_predicted,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# session parameters
sess = tf.InteractiveSession()
#initialising variables
init = tf.initialize_all_variables()
sess.run(init)
saver = tf.train.Saver()
save_path="/Users/Enkay/Documents/Viky/python/img-classification/mark2.ckpt"

In [ ]:
# grabbing the default graph
g = tf.get_default_graph()

# every operations in our graph
[op.name for op in g.get_operations()]

In [ ]:
epoch=5000
#change batch size according to your hardware's power. For GPU's use batch size in powers of 2 like 2,4,8,16...
batch_size=20 
previous_batch=0

In [ ]:
for i in range(epoch):
    #batch wise training 
    if previous_batch >= len(X_train) : #total --> total number of training images
        previous_batch=0    
    current_batch=previous_batch+batch_size
    x_input=X_train[previous_batch:current_batch]
    x_images=np.reshape(x_input,[batch_size,56,56,3])
    y_input=Y_train[previous_batch:current_batch]
    y_label=np.reshape(y_input,[batch_size,2])
    previous_batch=previous_batch+batch_size
    _,loss=sess.run([train_step, cross_entropy], feed_dict={x: x_images,y_: y_label}) 
    if i%500==0:
        n=50 #number of test samples
        # increase the number of test samples with higher RAM. if you have less RAM, limit your test sample or 
        # run test accross larger samples once in every 1000 epochs or so..  
        x_test_images=np.reshape(X_test[0:n],[n,56,56,3])
        y_test_labels=np.reshape(Y_test[0:n],[n,2])
        Accuracy=sess.run(accuracy,
                           feed_dict={
                        x: x_test_images ,
                        y_: y_test_labels
                      })
        print "Iteration no :{} , Accuracy:{} , Loss : {}" .format(i,Accuracy,loss)
        saver.save(sess, save_path, global_step = i)
    elif i % 100 ==0:   
        print "Iteration no :{} Loss : {}" .format(i,loss)

In [ ]:
x_input=X_val
x_images=np.reshape(x_input,[len(X_val),56,56,3])
y_input=Y_val
y_label=np.reshape(y_input,[len(Y_val),2])

Accuracy_validation=sess.run(accuracy,
                           feed_dict={
                        x: x_images ,
                        y_: y_label
                      })

In [ ]:
Accuracy_validation=round(Accuracy_validation*100,2)
print "Accuracy in the validation dataset: {} %" .format(Accuracy_validation)

In [ ]:
def process_img(img):
        img=img.resize((56, 56), Image.ANTIALIAS) #resize the image
        img = np.array(img)
        img=img/np.max(img).astype(float) 
        img=np.reshape(img, [1,56,56,3])
        return img


In [ ]:
#test your own images 
test_image=Image.open('/path to file')
test_image= process_img(test_image)
predicted_array= sess.run(y_predicted, feed_dict={x: test_image})
predicted_class= np.argmax(predicted_array)
if predicted_class==0:
    print "It is a cat"  
else :
    print "It is a dog  "